In [20]:
# 데이터 준비 1

from datasets import load_dataset

# klue_ynat_dataset = load_dataset('klue', 'ynat')
# klue_ynat_train = klue_ynat_dataset['train']
# klue_ynat_test = klue_ynat_dataset['validation']

klue_ynat_train = load_dataset('klue', 'ynat', split='train')
klue_ynat_test = load_dataset('klue', 'ynat', split='validation')

In [21]:
klue_ynat_train

Dataset({
    features: ['guid', 'title', 'label', 'url', 'date'],
    num_rows: 45678
})

In [16]:
klue_ynat_train.data
print( klue_ynat_train[0] )
klue_ynat_train.features['label']

{'guid': 'ynat-v1_train_00000', 'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영', 'label': 3, 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947', 'date': '2016.06.30. 오전 10:36'}


ClassLabel(names=['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'], id=None)

In [22]:
# 데이터 준비 2. 불필요한 컬럼 제거
klue_ynat_train = klue_ynat_train.remove_columns(['guid', 'url', 'date'])
klue_ynat_test = klue_ynat_test.remove_columns(['guid', 'url', 'date'])

In [23]:
klue_ynat_train[0]

{'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영', 'label': 3}

In [35]:
# 데이터 준비 3. label 숫자에 대한 문자열 컬럼 추가
print( klue_ynat_train.features["label"].names )
print( klue_ynat_train.features['label'].int2str(1) ) # 번호에 해당하는 위치의 데이터 반환

label_obj = klue_ynat_train.features['label']
def make_label_str(row):
    label_str = label_obj.int2str(row['label'])
    row['label_str'] = label_str
    return row

# klue_ynat_train2 = klue_ynat_train.map(make_label_str)
# klue_ynat_test2 = klue_ynat_test.map(make_label_str)

klue_ynat_train2 = klue_ynat_train.map(make_label_str, batched=True, batch_size=1000)
klue_ynat_test2 = klue_ynat_test.map(make_label_str, batched=True, batch_size=1000)

['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']
경제


Map:   0%|          | 0/45678 [00:00<?, ? examples/s]

Map:   0%|          | 0/9107 [00:00<?, ? examples/s]

In [37]:
klue_ynat_test2[0]

{'title': '5억원 무이자 융자는 되고 7천만원 이사비는 안된다', 'label': 2, 'label_str': '사회'}

In [43]:
# 데이터 준비 4. 훈련, 검증 분할

splitted_dataset = klue_ynat_train2.train_test_split(test_size=10000, shuffle=True, seed=42)
klue_ynat_train3 = splitted_dataset['train']
klue_ynat_valid3 = splitted_dataset['test']
klue_ynat_test3 = klue_ynat_test2

In [54]:
# 데이터 준비 5. 데이터 tokenize

from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "klue/roberta-base"


def tokenize_title(samples):
    tokenized_title = \
        tokenizer(samples["title"], padding="max_length", truncation=True)
    # samples['title'] = tokenize_title
    return tokenized_title
    
tokenizer = AutoTokenizer.from_pretrained(model_id)
train_dataset = klue_ynat_train3.map(tokenize_title, batch_size=True)
validation_dataset = klue_ynat_valid3.map(tokenize_title, batch_size=True)
test_dataset = klue_ynat_test3.map(tokenize_title, batch_size=True)

Map:   0%|          | 0/35678 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9107 [00:00<?, ? examples/s]

In [55]:
train_dataset

Dataset({
    features: ['title', 'label', 'label_str', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 35678
})

In [56]:
# 모델 준비

num_labels = len(klue_ynat_train.features['label'].names)
model = AutoModelForSequenceClassification.from_pretrained(model_id, 
                                                           num_labels=num_labels)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np

training_args = TrainingArguments(
    output_dir="./checkpoints2",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    push_to_hub=False
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    # processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

c:\Users\human\.conda\envs\human-dl-env2\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [60]:
trainer.train()

  0%|          | 0/4460 [00:00<?, ?it/s]

KeyboardInterrupt: 